<a href="https://colab.research.google.com/github/JBenjaminGM/ProyectoFinal-Analisis-de-Sentimiento/blob/main/Modelo_de_Entrenamiento_con_Bert_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Montamos Google Drive

In [1]:
#Montar Colab en Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Instalamos las bliebliotecas necesarias

In [2]:
# Biblioteca para procesamiento avanzado de lenguaje natural (NLP)
!pip install transformers==4.30

# Facilita el acceso y la manipulación de conjuntos de datos para entrenar modelos de aprendizaje automático
!pip install datasets

# Biblioteca de aprendizaje automático de código abierto para construir y entrenar modelos de redes neuronales
!pip install --upgrade tensorflow

# Optimiza el entrenamiento de modelos de aprendizaje profundo en PyTorch para un rendimiento mejorado
!pip install accelerate -U

# Extensión de transformers para utilizar modelos preentrenados en PyTorch, una biblioteca de aprendizaje automático de código abierto
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 2.4 MB/s eta 0:00:00


In [33]:
# Módulo para facilitar el entrenamiento de modelos y la gestión de experimentos
from transformers import Trainer, TrainingArguments

# Modelo preentrenado DistilBERT para clasificación de secuencias
from transformers import DistilBertForSequenceClassification

# Tokenizador rápido asociado a DistilBERT para un procesamiento eficiente de texto
from transformers import DistilBertTokenizerFast

# Utilidades para el manejo de datos durante el entrenamiento, como el relleno de secuencias
from transformers import DataCollatorWithPadding

# Módulo para la creación sencilla de pipelines que encapsulan tareas comunes de procesamiento de lenguaje natural
from transformers import pipeline

# Biblioteca para la carga eficiente de métricas para evaluar el rendimiento del modelo
from datasets import load_metric, Dataset

# Manipulación eficiente de operaciones
import numpy as np

# Herramientas para la codificación de etiquetas, útiles en tareas de clasificación
from sklearn.preprocessing import LabelEncoder

# Biblioteca para el manejo y análisis de datos tabulares
import pandas as pd

# Módulo para la manipulación de operaciones relacionadas con el sistema operativo
import os

# Biblioteca para optimizar el entrenamiento de modelos de aprendizaje profundo en PyTorch
import accelerate

Hacemos la lectura de los datos de entrenamiento

In [5]:
df = pd.read_csv('data_train.csv', delimiter=';', encoding='latin-1')

In [6]:
df.columns = ['utterance', 'label']

In [7]:
df['label'] = df['label'].replace({-1:0, 0:1, 1:2})

In [8]:
df

,utterance,label
0,"esa chamba está chévere, todos son buena onda.",2
1,"la comida de la abuela es la mejor, me llena e...",2
2,"no sé qué pasó, pero ya no me habla, estoy tri...",0
3,"ayer la pase bien en la reunión, risas y chist...",2
4,"el tráfico hoy estuvo infernal, llegué tarde a...",0
...,...,...
598,"la serie de ciencia ficción está al 1, trama i...",2
599,"qué bajón, se me cayó el helado justo antes de...",0
600,"la comida en la pizzería estuvo 1, masa crujie...",2
601,me crucé con mi ex en el centro y ni me saludó...,0


Lo transformamos en un Dataset para el entrenamiento

In [9]:
dataset = Dataset.from_pandas(df)

In [10]:
dataset = dataset.train_test_split(test_size=0.2) #Dividimos el conjunto de datos en train80 y test20

In [14]:
print(dataset['train'][0])

{'utterance': 'anoche la juerga estuvo 1, bailamos hasta el amanecer.', 'label': 2}


In [12]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-multilingual-cased') #Definimos el tokenizador de DistilBERT

(…)cased/resolve/main/tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

(…)ultilingual-cased/resolve/main/vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

(…)ingual-cased/resolve/main/tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

(…)tilingual-cased/resolve/main/config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

In [15]:
#Función simple para tokenizar por lotes expresiones con truncamiento
def preprocess_function(examples):
    return tokenizer(examples["utterance"], truncation=True)

In [16]:
seq_clf_tokenized = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/482 [00:00<?, ? examples/s]

Map:   0%|          | 0/121 [00:00<?, ? examples/s]

In [17]:
print(seq_clf_tokenized['train'][0])

{'utterance': 'anoche la juerga estuvo 1, bailamos hasta el amanecer.', 'label': 2, 'input_ids': [101, 12797, 11119, 10109, 23005, 69248, 10113, 25835, 122, 117, 48775, 21114, 10310, 11410, 10125, 28149, 54881, 10165, 119, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [20]:
# DataCollatorWithPadding crea un lote de datos. También rellena dinámicamente el texto con la
# longitud del elemento más largo en el lote, haciéndolos todos de la misma longitud.
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [22]:
unique_sequence_labels = list (set(seq_clf_tokenized['train']['label']))#Etiquetas unicas y ordenadas

In [23]:
unique_sequence_labels

[0, 1, 2]

In [25]:
sequence_clf_model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-multilingual-cased',
    num_labels=len(unique_sequence_labels),#Definimos el número de etiquetas
)

#Establecer un índice -> diccionario de etiquetas
sequence_clf_model.config.id2label = {i: l for i, l in enumerate(unique_sequence_labels)}

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bia

In [27]:
sequence_clf_model.config.id2label[0] #Primera etiqueta del conjunto de etiquetas

0

In [28]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred):  #Método personalizado para tomar logits y calcular la precisión del conjunto de evaluación
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

<ipython-input-28-ba1ec6707d82>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [29]:
os.makedirs("/content/drive/MyDrive/results", exist_ok=True)

In [35]:
epochs = 8

training_args = TrainingArguments(# Definimos los parametros basicos del entrenamiento
    output_dir="/content/drive/MyDrive/results",# Guardar putnos de control
    num_train_epochs=epochs,# Ciclos de entrenamiento con los datos completos
    per_device_train_batch_size=32, # Numero de secuencias que se cargaran por lote
    per_device_eval_batch_size=32,  # y que se procesarán en paralelo
    load_best_model_at_end=True, # Cargar el modelo con el mejor desempeño, evita el sobreentrenamiento

    # Número de pasos de calentamiento para el programador de tasa de aprendizaje
    warmup_steps=len(seq_clf_tokenized['train']) // 5,
    weight_decay = 0.05,

    logging_steps=1, # Mensaje log, despues de cada entrenamiento
    log_level='info', #  Determinar el nivel de log que se mostrarán en la terminal
    evaluation_strategy='epoch', # Determinar la estrategia de evaluación en paralelo
    eval_steps=50, # Numero de pasos de evaluación
    save_strategy='epoch' # Determinar la estrategia de guardado
)

# Define el trainer:

trainer = Trainer(
    model=sequence_clf_model, # Definimos el modelo que vamos a usar
    args=training_args, # Definimos los parámetros del entrenamiento
    train_dataset=seq_clf_tokenized['train'], # Definimos el conjunto de entrenamiento
    eval_dataset=seq_clf_tokenized['test'], # Definimos el conjunto de evaluación
    compute_metrics=compute_metrics, # Definimos el método de evaluación en este caso accuracy
    data_collator=data_collator # Definimos el data_collator
)

Found safetensors installation, but --save_safetensors=False. Safetensors should be a preferred weights saving format due to security and performance reasons. If your model cannot be saved by safetensors please feel free to open an issue at https://github.com/huggingface/safetensors!
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [36]:
# Evaluamos
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: utterance. If utterance are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 121
  Batch size = 32


{'eval_loss': 1.0941603183746338,
 'eval_accuracy': 0.4380165289256198,
 'eval_runtime': 5.7189,
 'eval_samples_per_second': 21.158,
 'eval_steps_per_second': 0.699}

In [37]:
# Entrenamos el modelo
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: utterance. If utterance are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 482
  Num Epochs = 8
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 128
  Number of trainable parameters = 135,326,979


Epoch,Training Loss,Validation Loss,Accuracy
1,1.054800,1.041477,0.487603
2,0.776700,0.980210,0.578512
3,0.450500,0.798830,0.628099
4,0.196800,0.497025,0.785124
5,0.029100,0.594035,0.809917
6,0.093300,0.445777,0.892562
7,0.009400,0.473643,0.867769
8,0.015100,0.450115,0.867769


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: utterance. If utterance are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 121
  Batch size = 32


Saving model checkpoint to /content/drive/MyDrive/results/checkpoint-16
Configuration saved in /content/drive/MyDrive/results/checkpoint-16/config.json
Model weights saved in /content/drive/MyDrive/results/checkpoint-16/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: utterance. If utterance are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 121
  Batch size = 32
Saving model checkpoint to /content/drive/MyDrive/results/checkpoint-32
Configuration saved in /content/drive/MyDrive/results/checkpoint-32/config.json
Model weights saved in /content/drive/MyDrive/results/checkpoint-32/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: utterance. If utteranc

TrainOutput(global_step=128, training_loss=0.45619998301481246, metrics={'train_runtime': 1045.269, 'train_samples_per_second': 3.689, 'train_steps_per_second': 0.122, 'total_flos': 23432141361132.0, 'train_loss': 0.45619998301481246, 'epoch': 8.0})

In [38]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: utterance. If utterance are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 121
  Batch size = 32


{'eval_loss': 0.44577673077583313,
 'eval_accuracy': 0.8925619834710744,
 'eval_runtime': 3.6126,
 'eval_samples_per_second': 33.494,
 'eval_steps_per_second': 1.107,
 'epoch': 8.0}

In [39]:
pipe = pipeline('sentiment-analysis', model=sequence_clf_model, tokenizer=tokenizer)
pipe('Hoy todo estuvo chévere, ni frío ni calor, día relax.')

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


[{'label': 1, 'score': 0.898716390132904}]

In [26]:
trainer.save_model("/content/drive/MyDrive/modeloDefinitivo/")

Saving model checkpoint to /content/drive/MyDrive/modeloDefinitivo/
Configuration saved in /content/drive/MyDrive/modeloDefinitivo/config.json
Model weights saved in /content/drive/MyDrive/modeloDefinitivo/pytorch_model.bin
